In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pyspark as spark

#read contents of csv into variables
datacenters = pd.read_csv("../data/datacenters.csv")
demand = pd.read_csv("../data/demand.csv")
selling_prices = pd.read_csv("../data/selling_prices.csv")
servers = pd.read_csv("../data/servers.csv")

# print(datacenters)
# print(demand)
# print(selling_prices)
# print(servers)

# First look at demand changes

In [ ]:
laten_sens_high = demand.loc[demand["latency_sensitivity"] == "high"]
laten_sens_med = demand.loc[demand["latency_sensitivity"] == "medium"]
laten_sens_low = demand.loc[demand["latency_sensitivity"] == "low"]

laten_sens_high_np = laten_sens_high.to_numpy()
laten_sens_med_np = laten_sens_med.to_numpy()
laten_sens_low_np = laten_sens_low.to_numpy()

#print(laten_sens_high_np)
fig, axs = plt.subplots(3, sharey=True)
fig.suptitle("cpuS1 demand over timesteps")
axs[0].plot(laten_sens_high_np[:, 0], laten_sens_high_np[:, 2], color="r")
axs[1].plot(laten_sens_med_np[:, 0], laten_sens_med_np[:, 2], color="b")
axs[2].plot(laten_sens_low_np[:, 0], laten_sens_low_np[:, 2], color="g")

# print(laten_sens_high)
# print(laten_sens_low)
# print(laten_sens_low)


In [ ]:
servergen_maintenance_dict = {}
servergen_totalcost_dict = {}
servergen_profit_dict = {}
for i in servers["server_generation"]:
    servergen_maintenance_dict[i] = []
    servergen_totalcost_dict[i] = []
    servergen_profit_dict[i] = []

In [ ]:
for i in servergen_maintenance_dict.keys():
    #calc maintenance cost over different timesteps using maintenance equation
    timestep_array = np.arange(1,96,1)
    maintenance_fee = servers[servers["server_generation"] == i]["average_maintenance_fee"].iloc[0]
    ts_array = 1.5 * timestep_array
    timestep_array = (1+ ts_array/96 * np.log2(ts_array/96)) * maintenance_fee
    timestep_array
    servergen_maintenance_dict[i] = timestep_array

    #calc totalcost
    servergen_energycost = servers[servers["server_generation"] == i]["energy_consumption"].iloc[0]
    servergen_totalcost_dict[i] = servergen_energycost + timestep_array

    #calc generated profit for that servergen
    server_capacity = servers[servers["server_generation"] == i]["capacity"].iloc[0]
    selling_price = selling_prices[selling_prices["server_generation"] == i]["selling_price"].mean()
    mean_revenue = np.full((95),selling_price*server_capacity)
    servergen_profit_dict[i] = mean_revenue

In [ ]:
fig, axs = plt.subplots(7, sharey=True)
fig.set_figheight(15)
fig.set_figwidth(10)
fig.suptitle("maintenance fee changes over server lifespan")
counter = 0
for i in servergen_maintenance_dict.keys():
    # if(counter > 3):
    #     counter += 1
    #     continue
    axs[counter].plot(np.arange(1,96,1), servergen_maintenance_dict[i], color="r")
    axs[counter].plot(np.arange(1,96,1), servergen_profit_dict[i], color="b")
    axs[counter].plot(np.arange(1,96,1), servergen_totalcost_dict[i], color="g")
    counter +=1

In [ ]:
#TLDR WILL ALWAYS GENERATE PROFIT NO MATTER HOW OLD SERVER GETS AND NO MATTER ENERGY PRICE only move action/having more
# than needed(server util is low) can make it generate negative profit

In [973]:
import numpy as np
from scipy.optimize import minimize

#for DC1 and servergen cpus1 over first TIMESTEPS timesteps

TIMESTEPS = 72

dc1_cap = datacenters[datacenters["datacenter_id"] == "DC1"]["slots_capacity"].iloc[0]
cpu_energies = servers["energy_consumption"].to_numpy()
purchase_prices = servers["purchase_price"].to_numpy()
capacity = servers["capacity"].to_numpy()

cpus1_energy= servers[servers["server_generation"] == "CPU.S1"]["energy_consumption"].iloc[0]
cpus1_purchasecost = servers[servers["server_generation"] == "CPU.S1"]["purchase_price"].iloc[0]
demand2 = demand[demand["latency_sensitivity"] == "low"]
demands = demand2.drop(columns=["latency_sensitivity","time_step"]).iloc[0:TIMESTEPS].to_numpy()
selling_prices_array = selling_prices[selling_prices["latency_sensitivity"] == "low"]["selling_price"].to_numpy()
maint_prices = servers["average_maintenance_fee"].to_numpy()

timestep_array = np.arange(1,96,1)
cpus1_maintenance_cost = servers[servers["server_generation"] == "CPU.S1"]["average_maintenance_fee"].iloc[0]
ts_array = 1.5 * timestep_array
maintenance_cost_array = np.empty((95,7))
for i in range(7):
    maintenance_cost_array[:,i] = (1+ ts_array/96 * np.log2(ts_array/96)) * maint_prices[i]
epsilon = 0.00000001

#where x is an array containing what servergen was bought at each timestep for all servergens
def capacity_constraint(x):
    x = np.reshape(x,(TIMESTEPS,7))
    total = 0
    #servernumber * slotsize to get slots occupied
    #for cpu
    occupied_slots = np.sum(x[:,0:4] * 2)
    #for gpu
    total = occupied_slots + np.sum(x[:,4:7] * 4)
    # #get total number of servers purchased
    # total = np.sum(x)
    # #servernumber * slotsize to get slots occupied
    # total = total * 2
    #constraint used cap has to be less than dc1_cap
    return dc1_cap - total

#get utilisation over the timesteps
def utilisation(x, y):
    #print(x)
    #array of utilisation of each server at eachtimestep
    util = []
    for i in range(TIMESTEPS):
        #array of bought servergens at this timestep
        servergen = x[i]
        #array of min(servergen_supply,demand) at this timestep for all servergens
        s_d_min = y[i]
        #get cumulative sum of number of servers to get total owned at each timestep
        cumsum = []
        total = epsilon
        #calc number of servers at each timestep and their cap
        s_d_sum = []
        sum = 0
        for j in range(len(servergen)):
            sum += s_d_min[j]
            cumsum.append((total + servergen[j]) * capacity[j])
            total = cumsum[j]
        #cumsum = np.cumsum(servergen)
        #get their capacity
        #get demand met for servergen i
        #get fraction of servergen utilised at each timestep
        util.append(sum)

    return util, cumsum

def lifespan(x,y):
    #get number of servers bought for timesteps (servergen doesnt matter)
    ts_sum = np.sum(x, axis=1)
    cumsum = np.cumsum(ts_sum)+epsilon
    life_spans = []
    for i in range(1,TIMESTEPS+1):
        multiplication_arr = np.arange(i,0,-1)
        #array[i-1] = np.divide(np.sum(np.multiply(ts_sum[0:i], multiplication_arr[0:i])), 96)
        sum = 0
        for j in range(i):
            m = ts_sum[j] * multiplication_arr[j]
            sum += m
        life_spans.append(sum)

    # ts1x = x[0]
    # ts2x = x[1]
    # ts1ls = (ts1x/96) / (ts1x +epsilon)
    # ts2ls = ((ts1x)/48 + ts2x/96)/ (ts1x+ts2x+epsilon)

    return life_spans, cumsum

def profit(x, y):
    #get cumulative sum of number of servers for all servergens
    revenues = []
    #get generated revenue at each timestep
    for i in range(TIMESTEPS):
        servergen = x[i, :]
        #get demand met
        supply = y[i,:]
        revenue = 0
        for j in range(7):
            revenue += supply[j] * selling_prices_array[j].astype("int") * capacity[j]
        revenues.append(revenue)
    
    #calc energycost for all servergens at dc1
    energy_costs = cpu_energies * 0.25

    timestep_costs = []
    for i in range(len(x)):
        #get servers that have been maintained (not new)
        maintained_servers = x[:i]
        #calc cost of new servers and add to overall cost at end
        new_cost = x[i] * np.rint((purchase_prices + energy_costs + maintenance_cost_array[i])).astype("int")
        new_cost = np.sum(new_cost)
        #calc energy + maintenance cost
        energy_and_maint = maintenance_cost_array[:i] + energy_costs
        energy_and_maint = np.rint(energy_and_maint).astype("int")
        #multiply corresponding servers with their cost to get total for servergen at each ts
        maint_cost = np.sum(np.multiply(maintained_servers, energy_and_maint[:i]))
        # if(maint_cost.size <= 0):
        #     maint_cost = np.zeros((7))
        timestep_costs.append(maint_cost + new_cost)

    profit = []
    for i in range(len(revenues)):
        profit.append(revenues[i]-timestep_costs[i])
    # print(revenues[0])
    # print(len(timestep_costs))
    return profit

    # ts1x = x[0]
    # ts2x = x[1]
    # ts1revenue = min(60*ts1x, 4000) * 10
    # ts2revenue = min(60*ts1x+60*ts2x, 8160) * 10

    # maint_cost = maintenance_cost(x)
    # ts1cost = (1500 + energycost + maint_cost[0]) * ts1x
    # ts2cost = (energycost + maint_cost[1]) * ts1x + (energycost + maint_cost[0]) * ts2x

    # ts1p = ts1revenue-ts1cost
    # ts2p = ts2revenue-ts2cost
    # return [ts1p, ts2p]

def maintenance_cost(x):
    return maintenance_cost_array[0:len(x)]
    # ts1x = x[0]
    # ts2x = x[1]

    # ts1maintenance = (1+ 1.5/96 * np.log2(1.5/96)) * cpus1_maintenance_cost
    # ts2maintenance = (1+ 3/96 * np.log2(3/96)) * cpus1_maintenance_cost
    # return [ts1maintenance, ts2maintenance]


def objective_func(x, y):
    x = np.reshape(x,(TIMESTEPS, 7))
    y = np.reshape(y,(TIMESTEPS,7))
    U, cumsum = utilisation(x,y)
    L, cumsum = lifespan(x,y)
    P = profit(x, y)
    # for i in range(len(P)):
    #     print(P[i])
    #     print(cumsum[i])
    objectives = []
    # for i in range(len(U)):
    #     print(U[i])
    #     print(L[i])
    #     print(U[i]+L[i])
    #     #print(np.multiply(U[i],L[i]))
    # print(len(P))
    objectives = P
    Objective = np.sum(objectives)
    return Objective
    # ts1o = utilisation(x)[0] * lifespan(x)[0] * profit(x)[0]
    # ts2o = utilisation(x)[1] * lifespan(x)[1] * profit(x)[1]
    # O = ts1o + ts2o
    # return O

In [975]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp

def solve_knapsack():
    # Create the solver
    solver = pywraplp.Solver.CreateSolver("GLOB")

    # Variables
    x = [solver.IntVar(0, int(dc1_cap/4), f'x{i}') for i in range(TIMESTEPS*7)]
    # y is the min(supply, demand) at each timestep for each server
    y = [solver.IntVar(0, int(dc1_cap/4), f'y{i}') for i in range(TIMESTEPS*7)]
    # z is the accumulated number of servers at each timestep
    #z = [solver.IntVar(0, int(dc1_cap/2), f'z{i}') for i in range(TIMESTEPS*7)]
    print("Number of variables =", solver.NumVariables())

    # Constraints
    #solver.Add(sum(weights[i] * x[i] for i in range(num_items)) <= capacity)
    for i in range(7):
        cumsum = 0
        counter = i
        for i in range(TIMESTEPS):
            cumsum += x[counter]
            counter = counter+7

    #solver.Add(capacity_constraint(x) >= 0)
    for i in range(7):
        total_x = 0
        counter = i
        for j in range(TIMESTEPS):
            total_x += x[counter]
            solver.Add(total_x <= dc1_cap)
            solver.Add(y[counter] <= demands[j][i])
            # print("y",y[counter])
            # print("x",total_x)
            solver.Add(y[counter] <= total_x)
            counter=counter+7
    #print("Number of constraints =", solver.NumConstraints())
    # for i in solver.constraints():
    #     print(i)
    #print(solver.constraints)

    # decision_builder = solver.Phase(
    # x, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

    # Objective
    solver.Maximize(objective_func(x, y))

    # Solve
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Total value =', solver.Objective().Value())
        for i in range(7):
            print(f'Item {i}: {x[i].solution_value()}')
    else:
        print('The problem does not have an optimal solution.')

if __name__ == '__main__':
    solve_knapsack()

W0000 00:00:1725231976.083553  778138 linear_solver.cc:662] Unrecognized solver type: GLOB


AttributeError: 'NoneType' object has no attribute 'IntVar'